This program will load the data needed for the analysis and reporting of the insights.

- [Import modules](#1)
- [Load zipped files](#2)
- [Load customers and transactions](#3)
- [Join tables](#4)
- [Save loaded tables](#5)

### <a id="1"></a>Import modules

In [1]:
## Import modules for the defined functions needed

# Load files from the system 
import zipfile, fnmatch, os
# Data reading and manipulation
import pandas as pd
# Defined functions in the functions.py script
import functions

In [2]:
# Directory for the project (under LS Direct folder)
mainDir = "../LS Direct/"

In [3]:
# List of files in mainDir
os.listdir(mainDir)

['Archives',
 'Dataload_AP.ipynb',
 'EDA_AP.ipynb',
 'functions.py',
 'Loaded_Tables',
 'LS\xa0Direct\xa0- Technical Assessment - Ankur Patel.pptx',
 'Technical_Data.zip',
 '__pycache__']

### <a id="2"></a>Load zipped files

In [4]:
# Open the zip file in read mode and extract it in the same path
Technical_Data_Path = mainDir + "Technical_Data.zip"
with zipfile.ZipFile(Technical_Data_Path, 'r') as zip_ref:
    zip_ref.extractall()

In [5]:
# Folder with the payload data 
Test_Data_Path = mainDir + "Test_Data/"
Test_Data_Path

'../LS Direct/Test_Data/'

In [6]:
# Load each files in the path, filter to .zip, and extract it to the current working directory
pattern = '*.zip'
for root, dirs, files in os.walk(Test_Data_Path):
    for filename in fnmatch.filter(files, pattern):
#         print(os.path.join(root, filename))
        zipfile.ZipFile(os.path.join(root, filename)).extractall(os.path.join(root, os.path.splitext(filename)[0]))

In [7]:
# Payload periods
payload = os.listdir(Test_Data_Path)
len(payload)

192

In [8]:
# Exclude the zip extensions
payload_unzip = [x for x in payload if "zip" not in x]
# Zip extensions
payload_zip = [x for x in payload if "zip" in x]

In [9]:
# Unzipped payload folders
len(payload_unzip)

96

In [10]:
# Sort the list in ascending order for the earliest 5 time periods
sorted(payload_unzip)[:5]

['payload_2022-01-01_to_2022-01-07.csv',
 'payload_2022-01-08_to_2022-01-14.csv',
 'payload_2022-01-15_to_2022-01-21.csv',
 'payload_2022-01-22_to_2022-01-28.csv',
 'payload_2022-01-29_to_2022-02-04.csv']

In [11]:
# Sort the list in descending order for the latest 5 time periods
sorted(payload_unzip, reverse=True)[:5]

['payload_2023-10-28_to_2023-11-03.csv',
 'payload_2023-10-21_to_2023-10-27.csv',
 'payload_2023-10-14_to_2023-10-20.csv',
 'payload_2023-10-07_to_2023-10-13.csv',
 'payload_2023-09-30_to_2023-10-06.csv']

- Side note: zipped files save disk space and are easier to transfer. It could be decided to remove the zipped files once the data is extracted.

In [12]:
# Store directories and filenames into a table
payload_dir = []
for folder in sorted(payload_unzip):
    payload_period_files = os.listdir(Test_Data_Path + folder)
    for file in sorted(payload_period_files):
        payload_dir.append((folder, file))

payload_files = pd.DataFrame(payload_dir, columns=['Folder', 'File'])
payload_files.head()

,Folder,File
0,payload_2022-01-01_to_2022-01-07.csv,customers_2022-01-01_to_2022-01-07.csv
1,payload_2022-01-01_to_2022-01-07.csv,transactions_2022-01-01_to_2022-01-07.csv
2,payload_2022-01-08_to_2022-01-14.csv,customers_2022-01-08_to_2022-01-14.csv
3,payload_2022-01-08_to_2022-01-14.csv,transactions_2022-01-08_to_2022-01-14.csv
4,payload_2022-01-15_to_2022-01-21.csv,customers_2022-01-15_to_2022-01-21.csv


In [13]:
# Filpath for the files
payload_files["Filepath"] = Test_Data_Path + payload_files["Folder"] + "/" + payload_files["File"]

In [14]:
payload_files.tail()

,Folder,File,Filepath
187,payload_2023-10-14_to_2023-10-20.csv,transactions_2023-10-14_to_2023-10-20.csv,../LS Direct/Test_Data/payload_2023-10-14_to_2...
188,payload_2023-10-21_to_2023-10-27.csv,customers_2023-10-21_to_2023-10-27.csv,../LS Direct/Test_Data/payload_2023-10-21_to_2...
189,payload_2023-10-21_to_2023-10-27.csv,transactions_2023-10-21_to_2023-10-27.csv,../LS Direct/Test_Data/payload_2023-10-21_to_2...
190,payload_2023-10-28_to_2023-11-03.csv,customers_2023-10-28_to_2023-11-03.csv,../LS Direct/Test_Data/payload_2023-10-28_to_2...
191,payload_2023-10-28_to_2023-11-03.csv,transactions_2023-10-28_to_2023-11-03.csv,../LS Direct/Test_Data/payload_2023-10-28_to_2...


### <a id="3"></a>Load customers and transactions

#### _Customers_

In [15]:
# Filter to customers' files
payload_customers_path = payload_files[payload_files["File"].str.contains("customers")]
# reset index to prevent errors in for loops, and remove the unnecessary index column
payload_customers_path = payload_customers_path.reset_index().drop("index", axis=1)
payload_customers_path.shape

(96, 3)

In [16]:
# Reading each customers csv and appending to a list
# Also check if data size error (in case of inconsistent tables during different payload periods)
Customers_tables = []
customers_error_cnt = 0
customers_error_files = []
customers_columns = ["Customer_ID","First Name","Last Name","Address","City","State_Abbr","Zip","Start_Date"]
for i in range(len(payload_customers_path)):
    Customers = pd.read_csv(payload_customers_path["Filepath"][i])
    # Checks if all tables have the same column (works in this example since they are same, but the shape would error otherwise)
    if all(Customers.columns == customers_columns):
        Customers_tables.append(Customers)
    else:
        customers_error_cnt += 1
        customers_error_files.append(payload_customers_path["Filepath"][i])
        print("Error: Inconsistent data size. Please check the following file:")
        print(payload_customers_path["Filepath"][i])
print("\nList of {} Customers tables".format(len(Customers_tables)))
print("Errors in {} tables".format(customers_error_cnt))


List of 96 Customers tables
Errors in 0 tables


In [17]:
# Merging the list of customers ta tables into a single dataframe
Customers = pd.concat(Customers_tables)
Customers.shape

(161006, 8)

In [18]:
Customers

,Customer_ID,First Name,Last Name,Address,City,State_Abbr,Zip,Start_Date
0,C84725947570,Elizabeth,Jones,947 James Greens,Scottland,PR,603.0,2022-01-01
1,C97761921457,Taylor,Bennett,2980 Salas Springs Apt. 814,Sanchezbury,NE,68040.0,2022-01-01
2,C85509443959,Amanda,Crawford,NaN,NaN,NaN,NaN,2022-01-01
3,C77919217553,Melissa,Martinez,72595 Tamara Circle Apt. 394,Robertfurt,NE,68094.0,2022-01-01
4,C94654194089,Stacy,Sanders,NaN,NaN,NaN,NaN,2022-01-01
...,...,...,...,...,...,...,...,...
731,C57654725789,Mark,Rivera,NaN,NaN,NaN,NaN,2023-10-30
732,C98296500535,Catherine,Hill,176 John Mountain,Victoriahaven,OK,73017.0,2023-10-30
733,C46169950639,Eric,Reed,702 Schaefer Parkways Suite 621,Lake Robert,OK,73018.0,2023-10-30
734,C70529012461,Tara,Farley,NaN,NaN,NaN,NaN,2023-10-30


In [19]:
# Check duplicated rows
Customers[Customers.duplicated(keep=False)]

,Customer_ID,First Name,Last Name,Address,City,State_Abbr,Zip,Start_Date


In [20]:
# Remove duplicates (if there were any duplicates)
Customers = Customers.drop_duplicates()
Customers.shape

(161006, 8)

In [21]:
# Data type of Customers columns
Customers.info()

<class 'pandas.core.frame.DataFrame'>
Index: 161006 entries, 0 to 735
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Customer_ID  161006 non-null  object 
 1   First Name   161006 non-null  object 
 2   Last Name    161006 non-null  object 
 3   Address      122727 non-null  object 
 4   City         122727 non-null  object 
 5   State_Abbr   122727 non-null  object 
 6   Zip          122727 non-null  float64
 7   Start_Date   161006 non-null  object 
dtypes: float64(1), object(7)
memory usage: 11.1+ MB


In [22]:
# Null values
Customers.isnull().sum()

Customer_ID        0
First Name         0
Last Name          0
Address        38279
City           38279
State_Abbr     38279
Zip            38279
Start_Date         0
dtype: int64

Checking the loaded Customers table, the customer's ID, first name, last name, and start date are not missing; however, around 23% of the customer addresses are missing. The loading also checked for consistent data size, columns, and printed error message otherwise.

#### _Transactions_

In [23]:
# Filter to transactions' files
payload_transactions_path = payload_files[payload_files["File"].str.contains("transactions")]
# reset index to prevent errors in for loops, and remove the unnecessary index column
payload_transactions_path = payload_transactions_path.reset_index().drop("index", axis=1)
payload_transactions_path.shape

(96, 3)

In [24]:
# Reading each transactions csv and appending to a list
# Also check if data size error (in case of inconsistent tables during different payload periods)
Transactions_tables = []
transactions_error_cnt = 0
transactions_error_files = []
for i in range(len(payload_transactions_path)):
    Transactions = pd.read_csv(payload_transactions_path["Filepath"][i])
    # check if there are 8 columns as expected (in case of inconsistent tables during different payload periods)
    if Transactions.shape[1] == 8:
        Transactions_tables.append(Transactions)
    else:
        transactions_error_cnt += 1
        transactions_error_files.append(payload_transactions_path["Filepath"][i])
        print("Error: Please check the following file:")
        print(payload_transactions_path["Filepath"][i])
print("\nList of {} Transactions tables".format(len(Transactions_tables)))
print("Errors in {} tables".format(transactions_error_cnt))

Error: Please check the following file:
../LS Direct/Test_Data/payload_2023-01-21_to_2023-01-27.csv/transactions_2023-01-21_to_2023-01-27.csv
Error: Please check the following file:
../LS Direct/Test_Data/payload_2023-01-28_to_2023-02-03.csv/transactions_2023-01-28_to_2023-02-03.csv
Error: Please check the following file:
../LS Direct/Test_Data/payload_2023-03-25_to_2023-03-31.csv/transactions_2023-03-25_to_2023-03-31.csv
Error: Please check the following file:
../LS Direct/Test_Data/payload_2023-04-22_to_2023-04-28.csv/transactions_2023-04-22_to_2023-04-28.csv
Error: Please check the following file:
../LS Direct/Test_Data/payload_2023-05-13_to_2023-05-19.csv/transactions_2023-05-13_to_2023-05-19.csv
Error: Please check the following file:
../LS Direct/Test_Data/payload_2023-06-24_to_2023-06-30.csv/transactions_2023-06-24_to_2023-06-30.csv
Error: Please check the following file:
../LS Direct/Test_Data/payload_2023-08-12_to_2023-08-18.csv/transactions_2023-08-12_to_2023-08-18.csv
Error:

In [25]:
# checking the size shows that it included 1-2 unnecessary index columns
for i in range(13):
    # error file included index
    Transactions = pd.read_csv(transactions_error_files[i])
    print(Transactions.shape)

(7079, 9)
(7184, 9)
(6696, 9)
(6832, 9)
(6945, 9)
(6519, 9)
(7449, 9)
(4862, 9)
(4175, 9)
(4602, 9)
(4080, 9)
(3919, 9)
(1894, 10)


In [26]:
Transactions = pd.read_csv(transactions_error_files[12])
Transactions.head()

,Unnamed: 0.1,Unnamed: 0,Date,Transaction_ID,Customer_ID,Department,Category,SKU,Price,Discount
0,0,1,2023-10-28,532685089571,C54783873038,Men,Shirts,Crew Neck Shirt,22.77,NaN
1,1,2,2023-10-28,48243694426,C52820349964,Men,Bottoms,Corduroy Pants,38.77,NaN
2,2,3,2023-10-28,48243694426,C52820349964,Men,Bottoms,Chinos,39.77,NaN
3,3,5,2023-10-28,48243694426,C52820349964,Men,Shirts,Flannel Shirt,35.77,NaN
4,4,6,2023-10-28,29702850787,C86240445650,Women,Tops,Tunic,29.77,NaN


In [27]:
# Below is the modification to the loading error that occured due to the columns inconsistency
# Select the useful columns from the loaded table to remove the index columns
transactions_columns = ["Date","Transaction_ID","Customer_ID","Department","Category","SKU","Price","Discount"]
Transactions_tables = []
for i in range(len(payload_transactions_path)):
    Transactions = pd.read_csv(payload_transactions_path["Filepath"][i])
    Transactions = Transactions[transactions_columns]
    Transactions_tables.append(Transactions)
print("List of {} Transactions tables".format(len(Transactions_tables)))

List of 96 Transactions tables


In [28]:
# Merging the list of transactions ta tables into a single dataframe
Transactions = pd.concat(Transactions_tables)
Transactions.shape

(655605, 8)

In [29]:
# Check duplicated rows
Transactions[Transactions.duplicated(keep=False)].shape

(183705, 8)

In [30]:
# Remove duplicates (~90000 rows)
Transactions = Transactions.drop_duplicates()
Transactions.shape

(559233, 8)

In [31]:
# Null values
Transactions.isnull().sum()

Date                   0
Transaction_ID         0
Customer_ID            0
Department             0
Category               0
SKU                    0
Price                  0
Discount          535621
dtype: int64

The loaded Transactions table didn't include any missing values for all the columns except for Discount, which included over 95% of missing values. The loading also checked for consistent data size, columns, and printed error message otherwise. There were 13 tables that included index columns which were excluded before merging the tables.

In [32]:
# QC the datasets

### <a id="4"></a>Join tables

In [33]:
Customer_Transactions = pd.merge(Customers, Transactions, on="Customer_ID")
Customer_Transactions.shape

(536885, 15)

In [34]:
Customer_Transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536885 entries, 0 to 536884
Data columns (total 15 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Customer_ID     536885 non-null  object 
 1   First Name      536885 non-null  object 
 2   Last Name       536885 non-null  object 
 3   Address         409097 non-null  object 
 4   City            409097 non-null  object 
 5   State_Abbr      409097 non-null  object 
 6   Zip             409097 non-null  float64
 7   Start_Date      536885 non-null  object 
 8   Date            536885 non-null  object 
 9   Transaction_ID  536885 non-null  int64  
 10  Department      536885 non-null  object 
 11  Category        536885 non-null  object 
 12  SKU             536885 non-null  object 
 13  Price           536885 non-null  float64
 14  Discount        22705 non-null   float64
dtypes: float64(3), int64(1), object(11)
memory usage: 61.4+ MB


### <a id="5"></a>Save loaded tables
- File directory
- Merged Customers
- Merged Transactions

The tables are loaded and saved separately. It can be used the analysis to get a better understanding of the customers' transactions.

In [35]:
# Create folder to save the loaded tables
output_path = mainDir + "Loaded_Data/"
os.mkdir(output_path)

In [36]:
# Save filepaths for easier access
payload_files.to_csv(output_path + "payload_files_directory.csv", index=False)

In [37]:
# Save merged Customers table
Customers.to_csv(output_path + "Customers.csv", index=False)

In [38]:
# Save merged Transactions table
Transactions.to_csv(output_path + "Transactions.csv", index=False)

In [39]:
# Save joined Customer_Transactions table for analysis
Customer_Transactions.to_excel(output_path + "Customer_Transactions.xlsx", index=False, sheet_name="Customer_Transactions")

Saving a Log file of this run for future reference or checking.

In [40]:
functions.save_notebook()
current_file = 'Dataload_AP.ipynb'
output_file = 'Dataload_AP.html'
functions.output_HTML(current_file, output_file)

### Dataload Compeleted